<a href="https://colab.research.google.com/github/sainijagjit/CS6910-A2/blob/main/Assignment_2_iNaturalist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install pytorch-lightning --quiet
# install weights and biases
!pip install wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 4.9 MB/s eta 0:00:00


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as L
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler, Subset
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import EarlyStopping,Callback,ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import wandb
from google.colab import userdata

In [6]:
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Deep Learning/inaturalist_12K/


Mounted at /content/drive


In [7]:
zip_path = "/content/drive/MyDrive/Deep Learning/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip

In [8]:
%cd inaturalist_12K/

/content/inaturalist_12K


In [9]:
wandb.login(key=userdata.get('wandb_key'))
wandb_project="CS6910 - Assignment 2"
wandb_entity="ns24z060"

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


**Loading & Splitting Dataset**

In [10]:
class DataModule(L.LightningDataModule):
    def __init__(self, batch_size, augment_data= True ,data_dir = './',num_workers=2):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.augment_data = augment_data
        self.num_workers=num_workers


        self.augmentation = transforms.Compose([
              transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
              transforms.RandomRotation(degrees=15),
              transforms.RandomHorizontalFlip(),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        self.transform = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])



    def prepare_data(self):
        self.train_dataset = ImageFolder(root='train')
        self.test_dataset = ImageFolder(root='val')
        print(f'No of classes {self.train_dataset.classes}')
        print(f'Train Samples {len(self.train_dataset)}')
        print(f'Test Samples {len(self.test_dataset)}')


    def setup(self, stage=None):
        self.test_dataset.transform = self.transform
        labels = self.train_dataset.targets
        train_indices, val_indices = train_test_split(
            range(len(self.train_dataset)),
            test_size=0.2,
            random_state=42,
            stratify=labels
        )

        # Checking representation of each class in val set
        class_names = self.train_dataset.classes
        class_counts = {class_name: 0 for class_name in class_names}
        for val_indice in val_indices:
            _, class_idx = self.train_dataset[val_indice]
            class_name = class_names[class_idx]
            class_counts[class_name] += 1
        # print('Validation Datased Class Distribution')
        # for class_name, count in class_counts.items():
        #     print(f"Class: {class_name}, No. of samples: {count}")
        self.val_dataset = Subset(self.train_dataset, val_indices)
        self.train_dataset = Subset(self.train_dataset, train_indices)
        # num_classes = len(class_names)
        # input_dims = self.test_dataset[0][0].shape
        # print(f'Input image shape ({input_dims[0]},{input_dims[1]},{input_dims[2]})')
        # print('-------------')
        # print('-------------')
        # print(f'No of classes {num_classes}')
        # print(f'Train Samples {len(self.train_dataset)}')
        # print(f'Val Samples {len(self.val_dataset)}')
        # print(f'Test Samples {len(self.test_dataset)}')

        # def imshow(img):
        #     img = img / 2 + 0.5
        #     npimg = img.numpy()
        #     plt.imshow(np.transpose(npimg, (1, 2, 0)))
        #     plt.show()

        # dataiter = iter(self.train_dataloader)
        # images, labels = next(dataiter)
        # imshow(torchvision.utils.make_grid(images))

        if self.augment_data:
          self.train_dataset.dataset.transform = self.augmentation
        self.val_dataset.dataset.transform = self.transform

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)


    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)


    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.num_workers)



In [11]:
class Net(nn.Module):
  def __init__(self,
                cnn_activation_function,
                fcc_activation_function,
                num_filters,
                filter_size,
                dense_neurons,
                batch_size,
                batch_norm=True,
                stride=2,
               dropout=0.5
                ):
      super().__init__()
      self.batch_size=batch_size
      cnn_layers = []
      for i in range(5):
        in_channels = 3 if i==0 else num_filters
        cnn_layers.extend([nn.Conv2d(in_channels, num_filters, kernel_size=filter_size),
                      getattr(nn, cnn_activation_function)(inplace=True),
                      ])
        if batch_norm:
          cnn_layers.append(nn.BatchNorm2d(num_filters,affine=True))
        cnn_layers.append(nn.MaxPool2d(kernel_size=filter_size,stride=stride))

      self.cnn_net = nn.Sequential(*cnn_layers)
      n_sizes = self._get_conv_output((3,224,224))
      dnn_layers = [
          nn.Flatten(),
          nn.Linear(n_sizes, dense_neurons),
          nn.Dropout(dropout),
          getattr(nn, fcc_activation_function)(inplace=True),
          nn.Linear(dense_neurons, 10)]
      self.dnn_net = nn.Sequential(*dnn_layers)

  def _get_conv_output(self, shape):
        input = torch.autograd.Variable(torch.rand(self.batch_size, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(self.batch_size, -1).size(1)
        return n_size

  def _forward_features(self, x):
        return self.cnn_net(x)

  def forward(self, x):
        x =  self.cnn_net(x)
        x =  self.dnn_net(x)
        return x

In [12]:
class MyCnnModel(L.LightningModule):
    def __init__(self,
                 cnn_activation_function = 'ReLU',
                 fcc_activation_function = 'ReLU',
                 loss_function='cross_entropy',
                 num_filters=32,
                 filter_size=3,
                 dense_neurons=1024,
                 learning_rate=0.01,
                 optimizer='Adam',
                 batch_size=64):
        super().__init__()
        self.save_hyperparameters()
        self.model = Net(cnn_activation_function,fcc_activation_function,num_filters,filter_size,dense_neurons,batch_size)
        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.loss_function = getattr(F, loss_function)
        self.learning_rate = learning_rate
        self.optimizer = optimizer

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
      loss,acc = self._common_step(batch,batch_idx)
      self.log('train_loss', loss, on_step=False, on_epoch=True, logger=True)
      self.log('train_acc', acc, on_step=False, on_epoch=True, logger=True)
      return loss

    def validation_step(self, batch, batch_idx):
      loss,acc =  self._common_step(batch,batch_idx)
      self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      return loss


    def test_step(self,batch,batch_idx):
      loss,acc =  self._common_step(batch,batch_idx)
      self.log('test_loss', loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      self.log('test_acc', acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      return loss

    def _common_step(self, batch, batch_idx):
      inputs, target = batch
      output = self.forward(inputs)
      loss = self.loss_function(output, target)
      acc = self.accuracy(output, target)
      return loss,acc

    def configure_optimizers(self):
      return getattr(torch.optim,self.optimizer)(self.parameters(), lr=self.learning_rate)

In [ ]:
class PrintCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        print("Training is started!")
    def on_train_end(self, trainer, pl_module):
        print("Training is done.")

batch_size=128
model = MyCnnModel(batch_size=batch_size)
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode='max')
wandb_logger = WandbLogger(project=wandb_project, job_type='train', log_model='all')
early_stop_callback = EarlyStopping(monitor="val_loss",patience=3)

trainer = L.Trainer(
    max_epochs=10,
    check_val_every_n_epoch=1,
    log_every_n_steps=1,
    logger=wandb_logger,
    # fast_dev_run=1,
    # limit_train_batches=0.01,
    # limit_val_batches=0.03125,
    callbacks=[PrintCallback(),checkpoint_callback,early_stop_callback])
dm = DataModule(batch_size=batch_size)
trainer.fit(model, dm)
trainer.test(model, dm, ckpt_path='best')
wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


No of classes ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Train Samples 9999
Test Samples 2000


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params
------------------------------------------------
0 | model    | Net                | 573 K 
1 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
573 K     Trainable params
0         Non-trainable params
573 K     Total params
2.295     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]